# data

In [2]:
import torch 
import importlib
import monotonic
import data_manager
import metrics
import utils
importlib.reload(utils)
import train as Train
from train import execute_model_evaluation
import train_config
from data_manager import DatasetManager
from train_config import FlatACLConfig, FlatDLinearConfig, FlatNaiveConfig, FlatPatchTSTConfig, FlatTimeMixerConfig
from dataclasses import replace

%load_ext autoreload
%autoreload 2
modules_to_reload_list = [
    data_manager,
    Train,
    train_config,
    monotonic,
    # data_manager, # Reloaded only once even if listed twice
    utils,
    # train_config, # Reloaded only once even if listed twice
    metrics
]

# Initialize the data manager
data_mgr = DatasetManager(device='cuda')

# Load a synthetic dataset
data_mgr.load_trajectory('lorenz', steps=24999, dt=1e-2, ) # 51999 36999
# SCALE = False

LorenzSystem initialized with method: rk4 on device: cuda

Dataset: lorenz (synthetic)
Shape: torch.Size([25000, 3])
Channels: 3
Length: 25000
Parameters: {'steps': 24999, 'dt': 0.01}

Sample data (first 2 rows):
tensor([[1.0000, 0.9800, 1.1000],
        [1.0106, 1.2389, 1.0820]], device='cuda:0')


# AB SWD

## FRIREN
Note: used 'convex' mixing, which has been proven precviously that it matters little to the original baseline.

### huber

In [3]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='convex', ### Here
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 74.2999, mae: 6.4853, huber: 6.0056, swd: 42.3418, ept: 32.1924
Epoch [1/50], Val Losses: mse: 59.6500

### huber + 0.1 SWD

In [6]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='convex', ### Here
    loss_backward_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    ablate_deterministic_y0=False, 

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 141.0142, mae: 8.4018, huber: 7.9166, swd: 14.0622, ept: 10.8112
Epoch [1/50], Val Losses: mse: 110.98

### huber + 0.5 SWD

In [20]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='convex', ### Here
    loss_backward_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
    ablate_deterministic_y0=False, 

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 236.0685, mae: 10.9817, huber: 10.4934, swd: 13.4349, ept: 4.3301
Epoch [1/50], Val Losses: mse: 154.7

### MSE

In [21]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='convex', ### Here
    loss_backward_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 76.2142, mae: 6.7053, huber: 6.2235, swd: 44.6189, ept: 27.1548
Epoch [1/50], Val Losses: mse: 61.2947

### MSE + 0.1 SWD

In [10]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='convex', ### Here
    loss_backward_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
    ablate_deterministic_y0=False, 

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 75.8453, mae: 6.6967, huber: 6.2146, swd: 37.6889, ept: 28.3173
Epoch [1/50], Val Losses: mse: 62.1306

### MSE + 0.5 SWD

In [22]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='convex', ### Here
    loss_backward_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
    ablate_deterministic_y0=False, 

)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 82.0629, mae: 7.0258, huber: 6.5421, swd: 23.9779, ept: 17.9927
Epoch [1/50], Val Losses: mse: 70.0927

## TimeMixer

### huber

In [8]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0]
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 68.5378, mae: 6.0891, huber: 5.6115, sw

### huber + 0.1 SWD

In [7]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.1, 0.0]
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 78.7088, mae: 6.4985, huber: 6.0192, sw

### huber + 0.5 SWD

In [9]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.5, 0.0]
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 89.1695, mae: 7.0887, huber: 6.6066, sw

### MSE

In [10]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.0, 0.0]
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 66.8765, mae: 6.2210, huber: 5.7406, sw

### MSE + 0.1 SWD

In [11]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.1, 0.0]
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 67.1197, mae: 6.2273, huber: 5.7468, sw

### MSE + 0.5 SWD

In [11]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.5, 0.0]
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 69.3558, mae: 6.3330, huber: 5.8518, sw

## PatchTST 

### huber  

In [12]:
utils.reload_modules([utils])
cfg_patch_tst = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0]
)
exp_patch_tst = execute_model_evaluation('lorenz', cfg_patch_tst, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 71.3631, mae: 6.3279, huber: 5.8480, sw

### huber + 0.1 SWD

In [8]:
utils.reload_modules([utils])
cfg_patch_tst = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.1, 0.0]
)
exp_patch_tst = execute_model_evaluation('lorenz', cfg_patch_tst, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 83.0614, mae: 6.7965, huber: 6.3152, sw

### huber + 0.5 SWD

In [13]:
utils.reload_modules([utils])
cfg_patch_tst = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.5, 0.0]
)
exp_patch_tst = execute_model_evaluation('lorenz', cfg_patch_tst, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 99.4185, mae: 7.6075, huber: 7.1231, sw

### MSE

In [14]:
utils.reload_modules([utils])
cfg_patch_tst = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.0, 0.0]
)
exp_patch_tst = execute_model_evaluation('lorenz', cfg_patch_tst, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 69.8124, mae: 6.4039, huber: 5.9226, sw

### MSE + 0.1 SWD

In [12]:
utils.reload_modules([utils])
cfg_patch_tst = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.1, 0.0]
)
exp_patch_tst = execute_model_evaluation('lorenz', cfg_patch_tst, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 70.1795, mae: 6.4190, huber: 5.9375, sw

### MSE + 0.5 SWD

In [4]:
utils.reload_modules([utils])
cfg_patch_tst = train_config.FlatPatchTSTConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    enc_in=data_mgr.datasets['lorenz']['channels'],
    dec_in=data_mgr.datasets['lorenz']['channels'],
    c_out=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
    loss_backward_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.5, 0.0]
)
exp_patch_tst = execute_model_evaluation('lorenz', cfg_patch_tst, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 72.3810, mae: 6.5229, huber: 6.0410, sw

## DLinear

### huber  

In [16]:
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 72.2890, mae: 6.3031, huber: 5.8260, sw

### huber + 0.1 SWD

In [9]:
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.1, 0.0],
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 97.3154, mae: 6.9975, huber: 6.5188, sw

### huber + 0.5 SWD

In [5]:
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.5, 0.0],
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 139.4832, mae: 8.2891, huber: 7.8061, s

### MSE

In [18]:
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.0, 0.0],
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 71.1245, mae: 6.4317, huber: 5.9511, sw

### MSE + 0.1 SWD

In [13]:
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.1, 0.0],
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 71.6243, mae: 6.4569, huber: 5.9759, sw

### MSE + 0.5 SWD

In [19]:
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
    loss_backward_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
    loss_validate_weights = [1.0, 0.0, 0.0, 0.5, 0.0],
)
exp = execute_model_evaluation('lorenz', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([17500, 3])
Shape of validation data: torch.Size([2500, 3])
Shape of testing data: torch.Size([5000, 3])
global_std.shape: torch.Size([3])
Global Std for lorenz: tensor([7.9152, 9.0134, 8.6069], device='cuda:0')
Train set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Validation set sample shapes: torch.Size([336, 3]), torch.Size([336, 3])
Test set data shapes: torch.Size([5000, 3]), torch.Size([5000, 3])
Number of batches in train_loader: 132
Batch 0: Data shape torch.Size([128, 336, 3]), Target shape torch.Size([128, 336, 3])

Data Preparation: lorenz
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 132
Validation Batches: 15
Test Batches: 34

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 75.4739, mae: 6.6581, huber: 6.1759, sw